In [4]:
import tensorflow as tf
from tensorflow.python.framework import ops
import numpy as np

In [5]:
from keras.datasets import mnist #Keras is used only to get MNIST data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
Y_train = Y_train.reshape(60000, 1).T 
b = np.zeros((60000, 10))
b[np.arange(60000), Y_train] = 1
Y_train = b.T #To convert to one hot matrix
print(Y_train.shape)
X_train = X_train.reshape(60000, 28, 28, 1)/255 #Divided by 255 to normalize the input data
print(X_train.shape)
Y_test = Y_test.reshape(10000, 1).T
b = np.zeros((10000, 10))
b[np.arange(10000), Y_test] = 1
Y_test = b.T
print(Y_test.shape)
X_test = X_test.reshape(10000, 28, 28, 1)/255
print(X_test.shape)

(10, 60000)
(60000, 28, 28, 1)
(10, 10000)
(10000, 28, 28, 1)


In [6]:
learning_rate = 0.01
m = 60000
batch_size = 128
batchno = m // batch_size
beta = 0.9

In [7]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1], name = "X")
Y = tf.placeholder(tf.float32, [None, 10], name = "Y")
W1 = tf.get_variable("W1", [4, 4, 1, 8], initializer = tf.contrib.layers.xavier_initializer(seed = 0)) #(f, f, ncprev, nc)
W2 = tf.get_variable("W2", [2, 2, 8, 16], initializer = tf.contrib.layers.xavier_initializer(seed = 0))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
Z1 = tf.nn.conv2D(X, W1, strides = [1, 1, 1, 1], padding = "SAME")
A1 = tf.nn.relu(Z1)
P1 = tf.nn.max_pool(A1, ksize = [1,8,8,1], strides = [1,8,8,1], padding = 'SAME') #ksize - kernel size
Z2 = tf.nn.conv2d(P1,W2, strides = [1,1,1,1], padding = 'SAME')
A2 = tf.nn.relu(Z2)
P2 = tf.nn.max_pool(A2, ksize = [1,4,4,1], strides = [1,4,4,1], padding = 'SAME')
P2 = tf.contrib.layers.flatten(P2) #used for flattening
Z3 = tf.contrib.layers.fully_connected(P2, 6, activation_fn=None) #To form a fully connected layer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(Z3, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


AttributeError: module 'tensorflow._api.v1.nn' has no attribute 'conv2D'

In [23]:
sess = tf.InteractiveSession() #So that .eval works
sess.run(tf.global_variables_initializer()) #To initialize the variables
for i in range(10):
        for j in range(batchno):
            begin = j * batch_size
            end = min(begin + batch_size, 60000) #To prevent tracebacks
            Xm = X_train[:, begin:end]
            Ym = Y_train[:, begin:end]
            feed_dict = {X:Xm, Y:Ym}
            _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: Xm, Y: Ym}) #_Upgraded values which are later not required are stored in _
        train_accuracy = accuracy.eval(feed_dict=feed_dict)
        print("Step %d, training batch accuracy %g %%"%(i, train_accuracy*100))
test_accuracy = accuracy.eval(feed_dict={X : X_test, Y : Y_test})
print("Test accuracy: %g %%"%(test_accuracy*100))


Step 0, training batch accuracy 99.2188 %
Step 1, training batch accuracy 99.2188 %
Step 2, training batch accuracy 99.2188 %
Step 3, training batch accuracy 100 %
Step 4, training batch accuracy 100 %
Step 5, training batch accuracy 100 %
Step 6, training batch accuracy 100 %
Step 7, training batch accuracy 100 %
Step 8, training batch accuracy 100 %
Step 9, training batch accuracy 100 %
Test accuracy: 96.84 %
